Source: http://hilpisch.com/Continuum_N_Body_Simulation_Numba_27072013.html#/5/1

In [3]:
import numpy as np

In [4]:
nParticles = 1000
particle = np.random.standard_normal((nParticles, 3))
particlev = np.zeros_like(particle)

(1000L, 3L)


In [5]:
import time

In [9]:
def nbody(particle, particlev):  # NumPy arrays as input
    t0 = time.time(); nSteps = 5; dt = 0.01
    for step in range(1, nSteps + 1, 1):
        for i in range(nParticles):
            Fx = 0.0; Fy = 0.0; Fz = 0.0
            for j in range(nParticles):
                if j != i:
                    # Differential is distance between particle i and j's
                    dx = particle[j,0] - particle[i,0]
                    dy = particle[j,1] - particle[i,1]
                    dz = particle[j,2] - particle[i,2]
                    drSquared = dx * dx + dy * dy + dz * dz
                    drPowerN32 = 1.0 / (drSquared + np.sqrt(drSquared))
                    Fx += dx * drPowerN32
                    Fy += dy * drPowerN32
                    Fz += dz * drPowerN32
                particlev[i, 0] += dt * Fx
                particlev[i, 1] += dt * Fy
                particlev[i, 2] += dt * Fz
        for i in range(nParticles):
            particle[i,0] += particlev[i,0] * dt
            particle[i,1] += particlev[i,1] * dt
            particle[i,2] += particlev[i,2] * dt
    return time.time() - t0

In [10]:
import numba as nb

In [12]:
nbody_nb = nb.autojit(nbody)
firstrun = nbody_nb(particle, particlev)
print firstrun

0.25
